In [ ]:
import torch
import cv2
print("GPU disponible:", torch.cuda.is_available())

!nvidia-smi

%pip install ultralytics

GPU disponible: True
Tue Jul  8 20:32:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   55C    P3             20W /   55W |    1014MiB /   8188MiB |     27%      Default |
|                                         |                        |                  N/A |
+--------------------------

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

results = model.train(
    data="vision/model/config.yaml",  
    epochs=150,
    imgsz=640,
    batch=8,
    perspective=0.0005,
    scale=0.4,
    translate=0.04,
    degrees=5,
    shear=1,
    hsv_s=0.5,
    hsv_v=0.3,
    flipud=0.2,
    patience=15,
    save_period=20,
    name="larc-model",
)

model.val()

In [ ]:
model = YOLO('')
results = model('')
annotated_img = results[0].plot()

cv2.imshow("YOLO inference", annotated_img)
cv2.waitKey(5000)
cv2.destroyAllWindows()